# Week 3: Spatial data visualization

Earlier this week we learned how to code for data visualization and focused on numerical data. Today, we will expand this understanding to spatial data and visualizations.

Data you will need for this notebook: `nyct2020.zip` and `citibike_stations.csv`. We will also use `201307_citibike_tripdata.csv` which we downloaded on Tuesday.


## Introduction to geopandas

Recall last week we learned about `pandas`, a powerful Python package for working with dataframes. This week we will work with `geopandas`, a geospatial extension of `pandas`. 

**Geopandas** is a python library that allows us to ingest, analyze, and map geospatial vector data. It combines what we have learned in the previous two classes: The tabular data analysis tools in **Pandas** with the geometry handling of shapely. Under the hood, it is using a python library called **fiona**, which handles all different kinds of spatial file formats, and **pyproj**, which manages our coordinate reference systems.

The main data structures in Geopandas are GeoDataFrames and GeoSeries, which are intended to mirror the Pandas DataFrame and Series structures. 

The key distinction in Geopandas is that we will always a column called `geometry` like so that contains the geometries related to each row: 
<figure class="image">
<img src="https://autogis-site.readthedocs.io/en/2019/_images/geodataframe.png" alt="drawing" width="500" style="display: block; margin: 0 auto"/>
 <figcaption><center>(From Automating GIS Processes)</figcaption>
</figure>

### The three components of a GeoPandas GeoDataFrame
To create a GeoDataFrame, we need three things:

1. a pandas *DataFrame (df)*
2. a *CRS* (coordinate reference system presented by EPSG code, e.g., "epsg: 4326"); (Find EPSG codes here: https://epsg.io/transform)
3. a shapely *geometry list* which defines the geometric object types of each observation, e.g., points, lines, or polygons.


In [ ]:
# We need to install geopandas and some dependancies
# You may need to close down your environment and restart it
!pip install geopandas shapely pyogrio pyproj geopy scipy mapclassify folium

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, Point
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import folium

In [ ]:
# Let's load a few data files

# Outlines of NYC census blocks 
nyc_outlines = gpd.read_file('nyct2020.zip')
nyc_outlines.head()

In [ ]:
# NYC Outlines describes has shape chacteristics and some census data. 
# Variables ending in "E" are estimates and variables ending in "M" are margins of error
nyc_outlines.iloc[25].head(25)

In [ ]:
# We can plot the outlines of the census tracts 
fig, ax = plt.subplots(figsize=(7,7))
nyc_outlines.plot(ax=ax)

In [ ]:
# We can also plot it interactively with .explore()
nyc_outlines.explore(tooltip=["GEOID", "BoroName", "MedIncomeE"]) #tooltip lets you hover over a shape and see column variables

### Let's explore the three conditions necessary for geopandas dataframe


In [ ]:
# 1. a pandas df. We can explore nyc_outlines like any pandas dataframe
nyc_outlines.sort_values(by='TotalHouse', ascending=False).head()

In [ ]:
# 2. a coordinate reference system
nyc_outlines.crs

In [ ]:
# 3. a geometry column
nyc_outlines["geometry"].head()

We can also start from a pandas dataframe (or dictionary, or list) and build a geodataframe. We just need to specify the crs and the geometry to go from `pandas` -> `geopandas`.

In [ ]:
# Let's load in our bike data again
citibike = pd.read_csv('201307_citibike_tripdata.csv')
citibike.head()

In [ ]:
# Our data has two locations, the start and the stop. Geopandas only allows you to have one geometry as the base for the dataframe
# Let's choose the start location for now.
# Remember the three things we need:
# 1. a pandas df: citibike

# 2. a crs: this is usually the crs when you see latitude/longitude values
crs = "EPSG:4326"

# 3. a geometry: we can create this from the lat/lon columns
geometry = gpd.points_from_xy(citibike['start station longitude'], citibike['start station latitude'])

# All together:
citibike_gdf = gpd.GeoDataFrame(citibike, 
                                geometry = geometry,
                                crs = crs)
# check crs
print(citibike_gdf.crs)
# view first 5 rows
citibike_gdf.head()

In [ ]:
## YOUR TURN 
## Plot only a unique list of the starting stations as a static map
citibike_gdf.drop_duplicates(subset='start station id').plot()

In [ ]:
## YOUR TURN
## Count the number of trips that start from each station
## Then plot it interactively, displaying this value


## Joining DataFrames

Often times we want to combine more than one dataframe together, this is called merging or joining. `pandas` and `geopandas` excel at this! 

There are a few ways we can do this: 

In [ ]:
# The first is to merge two or more dataframes vertically. This is called concatenating
# Any columns that match will fit together nicely, any others will add NaN values 
# in the other dataframe rows

# For example, let's say our citibike dataframe was split in two
top = citibike.loc[0:100, :]
bottom = citibike.loc[100:, :]
df = pd.concat([top, bottom], ignore_index=True)
df.head()

In [ ]:
# The second is to merge two or more dataframes horizontally. This is called merging or joining
# This is best when you have two dataframes in which you think some rows go together

# For example, let's look at a dataframe which has the actual names of the bike share stations
citibike_stations = pd.read_csv('citibike_stations.csv')
citibike_stations

There are couple of different types of joins: 
- `Left outer join`: In a LEFT OUTER JOIN (how='left'), we keep all rows from the left and duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the right if they intersect and lose right rows that don’t intersect. 
- `Right outer join`: In a RIGHT OUTER JOIN (how='right'), we keep all rows from the right and duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the left if they intersect and lose left rows that don’t intersect. 
- `Inner join` (this is the default setting): In an INNER JOIN (how='inner'), we keep rows from the right and left only where their binary predicate is True. We duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the right and left only if they intersect and lose all rows that do not.


In [ ]:
# We can merge this to our trip data to give each row the actual station name for the start and end station

citibike_withnames = pd.merge(left=citibike, right=citibike_stations, how='left', left_on='start station id', right_on='station id').drop('station id',axis=1)
citibike_withnames = citibike_withnames.rename(columns={'station name': 'start station name'})
citibike_withnames.head()

In geopandas, we can do a "spatial join" that joins values from two dataframes that intersect with eachother. 

For example, let's count how many citibike stations are in each census tract to get a sense of bikeshare accessibility by using a spatial join between our new `nyc_outlines` dataset and our `citibike_gdf` dataset. 

We are going to use  `gpd.sjoin(left_geoDF,right_geoDF)`. This function optionally takes as an input `how` to specify what type of spatial join. 

Similar to regular joins, there are a few **spatial** joins: 
- `Left outer join`: In a LEFT OUTER JOIN (how='left'), we keep all rows from the left and duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the right if they intersect and lose right rows that don’t intersect. **A left outer join implies that we are interested in retaining the geometries of the left.**
- `Right outer join`: In a RIGHT OUTER JOIN (how='right'), we keep all rows from the right and duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the left if they intersect and lose left rows that don’t intersect. **A right outer join implies that we are interested in retaining the geometries of the right.**
- `Inner join` (this is the default setting): In an INNER JOIN (how='inner'), we keep rows from the right and left only where their binary predicate is True. We duplicate them if necessary to represent multiple hits between the two dataframes. We retain attributes of the right and left only if they intersect and lose all rows that do not. **An inner join implies that we are interested in retaining the geometries of the left.**

In this case, we want to join `nyc_outlines` and `citibike_gdf` with a **left outer join** because we want to keep the hits between our census tracts and all the citibike stations. 

In [ ]:
# First we need to make sure our coordinate systems are the same
print(citibike_gdf.crs)
print(nyc_outlines.crs)

In [ ]:
# Let's convert both to EPSG: 2263 
citibike_gdf = citibike_gdf.to_crs(epsg=2263)
print(citibike_gdf.crs)

In [ ]:
# Let's make a new gdf of just the stations
citibike_stations_gdf = citibike_gdf.drop_duplicates('start station id')[['geometry', 'start station id']]
# Look at the output before we assign it a new vairable
gpd.sjoin(nyc_outlines, citibike_stations_gdf ,how='left')


As we can, see there are duplicate rows of the census tracts where each geometry has intersected with multiple stations.


In [ ]:
cts_with_stations = gpd.sjoin(nyc_outlines, citibike_stations_gdf ,how='left')
station_counts = pd.DataFrame(cts_with_stations.groupby('GEOID').count()['start station id'].reset_index()).rename(columns={'start station id':'citibike_stations'})
station_counts.sort_values(by='citibike_stations')

In [ ]:
# We can add this back in as a column with "merge"

nyc_outlines = nyc_outlines.merge(station_counts, on='GEOID', how='left')
nyc_outlines.head()

## Buffers and Distance Metrics

Two other important functions of geospatial dataframes are `buffer`s and `distance` metrics.

A `buffer` is just as it sounds, an expansion of the geometry that you have. You can apply a buffer to Point, LineString, or Polygon but the resulting geometry will always be a Polygon. 

In [ ]:
# Look at our citibike stations
citibike_stations_gdf.geometry.iloc[0]

In [ ]:
# This will add 100 units around our geometry. In this CRS, that is feet. 
citibike_stations_gdf.buffer(100).iloc[0]

Importantly, you can also calculate the distance between two objects. 

In [ ]:
# For example, the distance from one station to all of the others
# Again, this will be in the units of geometry, so we usually calucate distance in meters or feet crs
reference_station = citibike_stations_gdf.iloc[0]
citibike_stations_gdf.distance(reference_station.geometry)

In [ ]:
# You can also check if two objects interest each other. Note that two Points will only intersect if they are identical
# If you have two Polygons, it will return True if any part of the geometries overlap at all
# This is the long form way of the sjoin. We can see which census tracts intersect with our reference station
nyc_outlines.intersects(reference_station.geometry)

In [ ]:
# We can see which one(s) are True
nyc_outlines[nyc_outlines.intersects(reference_station.geometry)]

In [ ]:
# Finally we can also check "contains". This is stricter than intersects
# This will only return True if the containing object fully engulfs the second object

# For Points, this is usually equivalent to intersects unless the point falls exactly on the exterior boundary of the Polygon
nyc_outlines[nyc_outlines.contains(reference_station.geometry)]

In [ ]:
## YOUR TURN 
## Count how many trips END in each census tract using spatial joins 


## Choropleth Maps

In [ ]:
# Let's look at our columns in the NYC census tract data again
nyc_outlines.columns

In [ ]:
# Let's create a new column, the percent of people who drive to work alone (no carpooling)
nyc_outlines['percent_drove'] = nyc_outlines['DroveAlone'] / nyc_outlines['CommutersE']

# We can plot one of our variables as the color, with "column="
fig, ax = plt.subplots(figsize=(10,10))
nyc_outlines.plot(column='percent_drove', ax=ax, legend=True, legend_kwds={'shrink': 0.75})

In [ ]:
# We can also color by categorical variables. One we have now is "BoroName" (borough)
fig, ax = plt.subplots(figsize=(10,10))
nyc_outlines.plot(column='BoroName', cmap='tab20', categorical=True, ax=ax, legend=True)

In [ ]:
# Plot census tracts by number of citibike stations in July 2013
fig, ax = plt.subplots(figsize=(10,10))
nyc_outlines.plot(column='citibike_stations', ax=ax)

### Saving Nice Plots

In [ ]:
# Finally, we can save these plots out so they look presentable:

# map tracts as a basemap 
ax = nyc_outlines.plot(facecolor="#aaaaaa", edgecolor="w", lw=0.5, figsize=(12, 9), legend=False)

# Plot Percent Drove by Quantiles
ax = nyc_outlines.plot(ax=ax, legend=True, cmap="plasma", column="MedIncomeE", scheme="Quantiles")
ax.axis("off") # Remove the lat/lon axes labels
ax.set_title("Median Income by Census Tract in 2020 Inflation Adjusted USD")
#save fig. dpi = resolution of pixels, bbox_inches='tight' means remove excess whitespace around the edges
ax.get_figure().savefig("nyc_medincome.png", dpi=300, bbox_inches="tight")

In [ ]:
## YOUR TURN 
## Create a map from one of the datasets we have used so far. What variables can you compare?
## Try to change the colormap 

